In [1]:
!pip install -q gradio
!pip install huggingsound
!pip install unidecode

In [2]:
# cargamos el modelo de huggingsound
import re
from huggingsound import SpeechRecognitionModel

# creación del modelo
model = SpeechRecognitionModel("patrickvonplaten/wav2vec2-large-xlsr-53-spanish-with-lm")

05/01/2022 23:15:56 - INFO - huggingsound.speech_recognition.model - Loading model...


/usr/local/lib/python3.7/dist-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:359: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [3]:
import gradio as gr

bloque = ''
codigo = None
indentacion = 0
# Esto para dar seguimiento al eliminado de una linea
linea_codigo = 0

In [4]:
def sim_jac(s1, s2):

    bigrams_s1 = []
    bigrams_s2 = []

    for i in range(len(s1) - 1):
        bigrams_s1.append(s1[i:i+2])
    
    for i in range(len(s2) - 1):
        bigrams_s2.append(s2[i:i+2])

    c_common = 0

    for i in bigrams_s1:
        if bigrams_s2.count(i) > 0:
            c_common += 1

    return c_common / ((len(s1) - 1) + (len(s2) - 1) - c_common)

In [44]:
def plantilla_funcion(instruccion):
    global indentacion

    before_keyword, keyword, after_keyword = instruccion.partition('nombre')
    recomendacion = ''
    output = ''

    if len(after_keyword) == 0:
        recomendacion = f'¡No me dijiste el nombre de la función!'
    else:
        # Obtenemos el nombre de la función por el usuario
        name_func = after_keyword.split(' ')[1]

        # Verificamos si desea poner parametros                                
        if instruccion.strip().split(' ')[-1] == name_func:
            parametros = ''
        else:
            before_keyword, keyword, after_keyword = instruccion.partition('parametros')


            if len(after_keyword) == 0:
                parametros = ''
                recomendacion = f'¡No me dijiste el nombre de los parámetros!'
            else:
                parametros = after_keyword.split(' ')[1:]
                parametros = ', '.join(parametros)

        if not recomendacion or recomendacion == '¡No me dijiste el nombre de los parámetros!':
            indentacion += 1

        output = f'def {name_func}({parametros}):\n' + '\t' * indentacion

    return output, recomendacion

In [48]:
plantilla_funcion('definir función con nombre magica y parametros x y z')[0]

'def magica(x, y, z):\n\t\t\t'

In [49]:
def plantilla_clase():
    global indentacion
    # ....

    indentacion += 1
    pass

def plantilla_variable():
    ''' Esta se encargara tambien de analizar los tipos entero

    Test
    >>> Definir variable con nombre i igual a numero 1 (int)
    >>> Definir variable con nombre i igual a numero 3.5 (float) 
    >>> Definir variable con nombre i igual a cadena hola (string) 
    >>> Definir variable con nombre i igual a lista/dic (string)   
    '''    
    pass   


In [39]:
import unidecode

def main(instruccion):
    global bloque
    # Todas las frases inician con definir

    # Obtenemos el bloque que será del contexto
    # Porque son los unicos que va modificar la identacion
    # bloque = {funcion, clase, estructura_control}
    bloque = instruccion.split(' ')[1]   

    # lista de comandos disponibles
    # []: Todo lo que esta corchetes son los valores que vamos obtener
    comandos = [
                'funcion',  # con nombre [nameFuc](y parámetros [x y z])
                'variable', # con nombre [nombre_variable] igual a /objeto_basico/
                'cadena',
                'numero',
                'lista',
                'diccionario',                                                       
                ]

    # Lista de funciones que nos devolveran las plantillas
    plantillas = [plantilla_funcion]

    # buscar el comando correspondiente más cercano al deseado
    scores = []
    ind_max_score = 0

    # similitud de Jaccard para las primeras dos palabras de la grabación
    for i, comando in enumerate(comandos):
        scores.append(sim_jac(instruccion.split(' ')[1], comando))

        # guardamos índice de la instrucción con más similitud
        if scores[-1] > scores[ind_max_score]:
            ind_max_score = i
    
    return plantillas[ind_max_score](instruccion)

In [8]:
def agregar_adentro(codigo, transcipcion):
    codigo2, recomendacion = main(transcipcion)
    return codigo + codigo2, recomendacion

In [45]:
# Variables globales
bloque = ''
codigo = None
indentacion = 0
linea_codigo = 0 # Esto para dar seguimiento al eliminado de una linea

def transcribe(audio, state=""):
    global bloque
    global codigo

    transcriptions_es = model.transcribe([audio])[0]
    # quitamos el acento de la transcripcion
    frase = unidecode.unidecode(transcriptions_es['transcription'])
    
    if not bloque:
        # Significa que es la primera vez
        codigo, recomendacion = main(frase)
        
    else:        
        if bloque == 'funcion':
            codigo, recomendacion = agregar_adentro(codigo, frase)
        elif bloque == 'clase':
            pass
    
    
    # Prueba
    # codigo = transcriptions_es['transcription']
    # recomendacion = "hola"
    return codigo, recomendacion

inputs = gr.inputs.Audio(label="Dar click para escuchar tu voz", type="filepath", source="microphone")
output1 = gr.outputs.Textbox(label="Asi se ve tu código")
output2 = gr.outputs.Textbox(label="Sugerencia")

title = "Expresate con voz"
description = "Aplicación que ayuda a programar a traves de tu voz"
examples = ['definir función con nombre magica y parámetros x y z','definir función', 'definir variable', 'definir clase']
article = "<a  style='color:#eb9f59;' href = 'https://github.com/gandres-dev/Hackaton-Common-Voice'> Repositorio de la app"
demo = gr.Interface(fn=transcribe, inputs=inputs, outputs=[output1,output2],
                    title=title, description=description, article=article,
                    allow_flagging="never", theme="darkpeach", examples=examples,
                    # live=True
                    )

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.7/dist-packages/gradio/interface.py:317: UserWarning: 'darkpeach' theme name is deprecated, using dark-peach instead.
  f"'{theme}' theme name is deprecated, using {DEPRECATED_THEME_MAP[theme]} instead."


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
Running on public URL: https://47464.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


## Probando la aplicación con widget

In [11]:
!pip install ipywebrtc

     |████████████████████████████████| 260 kB 19.3 MB/s 


In [12]:
from google.colab import output
output.enable_custom_widget_manager()

Crea el widget de audio

In [13]:
from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

Reescribir audio ahora en formato .wav

In [14]:
# Escribe el formato en webm
with open('test_audio.webm', 'wb') as f:
    f.write(recorder.audio.value)

# Convierte en webm -> wav
!ffmpeg -i test_audio.webm -ac 1 -f wav test_audio.wav -y -hide_banner -loglevel panic

## Modelos

In [15]:
audio_paths = ["test_audio.wav"]

# Obtenemos la frase del usuario
transcriptions_es = model.transcribe(audio_paths)[0]
# Obtener la frase
transcriptions_es['transcription']

  0%|          | 0/1 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [ ]:
main(transcriptions_es['transcription'])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17740a58-e960-4de3-bb66-1575286930f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>